### Additions in Mistral model compared ot the original transformer:- 

1. Rotary Positional Embeddings
2. SwiGLU activation function
3. RMS Norm
5. Grouped Query attention 
6. **Sliding window attention**

6. KV Caching :- 
    - **Rolling buffere cache**
    - **Prefilling and chunking**

#### Specific to Mixtral 8x7B, the feedforward networks are modified to get the mixtrue of experts output